<h1>Table of Contents<span class="tocSkip"></span></h1>


# Introduction
<hr style="border:2px solid black"> </hr>


**What?** Pandas pivot and unstack table



# Imports
<hr style="border:2px solid black"> </hr>

In [2]:
import numpy as np
import pandas as pd

# Read-in the dataset
<hr style="border:2px solid black"> </hr>

In [3]:
df = pd.read_excel('./sales-funnel.xlsx')
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [4]:
df.shape

(17, 8)

# Pivot the Data
<hr style="border:2px solid black"> </hr>


- Most people likely have experience with pivot tables in **Excel**. Pandas provides a similar function.
- The simplest pivot table must have a dataframe and an `index`, which stands for the column that the data will be aggregated upon and `values`, which are the aggregated value.
- In **simple terms**, it aggregates value and present them in a way that is easy to read.



In [4]:
df.pivot_table(index = ['Manager', 'Rep'], values = ['Price'])

Price
Manager       Rep                        
Debra Henley  Craig Booker   20000.000000
              Daniel Hilton  38333.333333
              John Smith     20000.000000
Fred Anderson Cedric Moss    27500.000000
              Wendy Yule     44250.000000


- By default, the values will be averaged, but we can do a count or a sum by providing the aggfun parameter.



In [5]:
# you can provide multiple arguments to almost every argument of the pivot_table function
df.pivot_table(index = ['Manager', 'Rep'], values = ['Price'], aggfunc = [np.mean, len])

mean   len
                                    Price Price
Manager       Rep                              
Debra Henley  Craig Booker   20000.000000     4
              Daniel Hilton  38333.333333     3
              John Smith     20000.000000     2
Fred Anderson Cedric Moss    27500.000000     4
              Wendy Yule     44250.000000     4


- If we want to see sales broken down by the products, the columns variable allows us to define one or more columns. 
- Note: The confusing points with the pivot_table is the use of columns and values. 
- Columns are optional - they provide an additional way to segment the actual values you care about. 
- The aggregation functions are applied to the values you've listed. 



In [6]:
df.pivot_table(index = ['Manager','Rep'], values = ['Price'],
               columns = ['Product'], aggfunc = [np.sum])

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

The NaNs are a bit distracting. If we want to remove them, we could use `fill_value` to set them to 0.

In [7]:
df.pivot_table(index = ['Manager', 'Rep'], values = ['Price', 'Quantity'],
               columns = ['Product'], aggfunc = [np.sum], fill_value = 0)

sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0


- You can move items to the index to get a different visual representation. 
- The following code chunk removes Product from the columns and add it to the index and also uses the margins = True parameter to add totals to the pivot table. 



In [8]:
df.pivot_table(index = ['Manager', 'Rep', 'Product'],
               values = ['Price', 'Quantity'], aggfunc = [np.sum], margins = True)

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2
All                                      522000       30


- We can define the status column as a category and set the order we want in the pivot table. 



In [9]:
df['Status'] = df['Status'].astype('category')
df['Status'] = df['Status'].cat.set_categories(['won', 'pending', 'presented', 'declined'])
df.pivot_table(index = ['Manager', 'Status'], values = ['Price'],
               aggfunc = [np.sum], fill_value = 0, margins = True)

sum
                          Price
Manager       Status           
Debra Henley  won         65000
              pending     50000
              presented   50000
              declined    70000
Fred Anderson won        172000
              pending      5000
              presented   45000
              declined    65000
All                      522000


- A really handy feature is the ability to pass a dictionary to the aggfunc so you can perform different functions on each of the values you select. 
- This has a side-effect of making the labels a little cleaner.



In [10]:
table = df.pivot_table(index = ['Manager','Status'], 
                       columns = ['Product'], 
                       values = ['Quantity','Price'],
                       aggfunc = {'Quantity': len, 'Price': [np.sum, np.mean]}, 
                       fill_value = 0)
table

Price                                       \
                          mean                                  sum   
Product                    CPU Maintenance Monitor Software     CPU   
Manager       Status                                                  
Debra Henley  won        65000           0       0        0   65000   
              pending    40000        5000       0        0   40000   
              presented  30000           0       0    10000   30000   
              declined   35000           0       0        0   70000   
Fred Anderson won        82500        7000       0        0  165000   
              pending        0        5000       0        0       0   
              presented  30000           0    5000    10000   30000   
              declined   65000           0       0        0   65000   

                                                     Quantity              \
                                                          len               
Product                 Maintenance Monitor Software      CPU Maintenance   
Manager       Status                                                        
Debra Henley  won                 0       0        0        1           0   
              pending         10000       0        0        1           2   
              presented           0       0    20000        1           0   
              declined            0       0        0        2           0   
Fred Anderson won              7000       0        0        2           1   
              pending          5000       0        0        0           1   
              presented           0    5000    10000        1           0   
              declined            0       0        0        1           0   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Debra Henley  won             0        0  
              pending         0        0  
              presented       0        2  
              declined        0        0  
Fred Anderson won             0        0  
              pending         0        0  
              presented       1        1  
              declined        0        0


- Once you have generated your data, it is in a DataFrame so you can filter on it using your standard DataFrame functions. e.g. We can look at all of our pending and won deals.



In [11]:
# .query uses strings for boolean indexing and we don't have to 
# specify the dataframe that the Status is comming from
table.query("Status == ['pending','won']")

Price                                                   \
                        mean                                  sum               
Product                  CPU Maintenance Monitor Software     CPU Maintenance   
Manager       Status                                                            
Debra Henley  won      65000           0       0        0   65000           0   
              pending  40000        5000       0        0   40000       10000   
Fred Anderson won      82500        7000       0        0  165000        7000   
              pending      0        5000       0        0       0        5000   

                                       Quantity                               
                                            len                               
Product               Monitor Software      CPU Maintenance Monitor Software  
Manager       Status                                                          
Debra Henley  won           0        0        1           0       0        0  
              pending       0        0        1           2       0        0  
Fred Anderson won           0        0        2           1       0        0  
              pending       0        0        0           1       0        0

# Unstack table
<hr style="border:2px solid black"> </hr>


- Pivot a level of the (necessarily hierarchical) index labels.



In [3]:
index = pd.MultiIndex.from_tuples([('one', 'a'), ('one', 'b'),

                                   ('two', 'a'), ('two', 'b')])

In [4]:
s = pd.Series(np.arange(1.0, 5.0), index=index)

In [5]:
s

one  a    1.0
     b    2.0
two  a    3.0
     b    4.0
dtype: float64

In [6]:
s.unstack(level=-1)

,a,b
one,1.0,2.0
two,3.0,4.0


In [7]:
s.unstack(level=0)

,one,two
a,1.0,3.0
b,2.0,4.0


In [9]:
df = s.unstack(level=0)
df.unstack()

one  a    1.0
     b    2.0
two  a    3.0
     b    4.0
dtype: float64

In [10]:
# Inverse operation
df.stack()

a  one    1.0
   two    3.0
b  one    2.0
   two    4.0
dtype: float64

# Pivot vs. Stack vs. Usntack
<hr style="border:2px solid black"> </hr>


- `DataFrame.pivot`   = Pivot a table based on column values.
- `DataFrame.stack`   = Pivot a level of the column labels (inverse operation from unstack).
- `DataFrame.unstack` = Pivot a level of the (necessarily hierarchical) index labels.



# References
<hr style="border:2px solid black"> </hr>


- [This notebook](https://github.com/ethen8181/machine-learning/blob/master/python/pivot_table/pivot_table.ipynb)
- [Blog: Pandas pivot table explained](http://pbpython.com/pandas-pivot-table-explained.html)
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html
    
